
# Cloud Memory Operations with Mem0 MemoryClient

This example demonstrates how to use Mem0's cloud-based MemoryClient for managing conversational memory and user preferences with both synchronous and asynchronous operations.

## Overview

This example showcases cloud-based memory management operations where we:

1. **Initialize MemoryClient instances** for both sync and async cloud operations
2. **Store conversation history** in the cloud with rich metadata
3. **Perform concurrent operations** using async/await patterns
4. **Search and filter memories** using natural language and structured queries

By using the cloud-based MemoryClient with async operations, you can leverage Mem0's managed infrastructure while performing multiple memory operations simultaneously. This is ideal for production applications that need scalable memory management without managing local storage.


## Setup and Imports

First, we'll import the necessary libraries for working with Mem0's cloud-based memory management system. We'll use both synchronous and asynchronous clients to demonstrate different usage patterns.


# Install the required dependencies:
%pip install agentops
%pip install mem0ai
%pip install python-dotenv

In [95]:
# Install the required dependencies:
%pip install agentops
%pip install mem0ai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [96]:

from mem0 import MemoryClient, AsyncMemoryClient
import agentops
import os
import asyncio
from dotenv import load_dotenv

## Environment Configuration

Load environment variables including API keys for AgentOps, Mem0, and OpenAI. These credentials are required for authenticating with the respective services.


In [97]:
# Load environment variables
load_dotenv()
os.environ["AGENTOPS_API_KEY"] = os.getenv("AGENTOPS_API_KEY")
mem0_api_key = os.getenv("MEM0_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Sample Data Setup

Define user identifiers and sample data that will be used throughout the demonstration. This includes user IDs for tracking memory ownership and agent/session identifiers for context.


In [98]:
# Sample user data for demonstration
user_id = "alice_demo"
agent_id = "assistant_demo"
run_id = "session_001"

# Sample conversation data demonstrating preference discovery through dialogue
sample_messages = [
    {"role": "user", "content": "I'm planning to watch a movie tonight. Any recommendations?"},
    {"role": "assistant", "content": "How about a thriller? They can be quite engaging."},
    {"role": "user", "content": "I'm not a big fan of thriller movies but I love sci-fi movies."},
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

# Sample user preferences representing various personal attributes
sample_preferences = [
    "I prefer dark roast coffee over light roast",
    "I exercise every morning at 6 AM",
    "I'm vegetarian and avoid all meat products",
    "I love reading science fiction novels",
    "I work in software engineering",
]

## Synchronous Memory Operations

The following function demonstrates how to use the synchronous MemoryClient for sequential cloud memory operations. This approach is straightforward but operations execute one after another.


In [99]:
def demonstrate_sync_memory_client(sample_messages, sample_preferences, user_id):
    """
    Demonstrate synchronous MemoryClient operations with cloud storage.
    
    This function performs sequential cloud memory operations including:
    - Initializing cloud-based memory client with API authentication
    - Adding conversation messages to cloud storage
    - Storing user preferences with metadata
    - Searching memories using natural language
    - Retrieving memories with filters
    - Cleaning up cloud memories
    
    """
    agentops.start_trace("mem0_memoryclient_sync_example",tags=["mem0_memoryclient_example"])
    try:
        # Initialize sync MemoryClient with API key for cloud access
        client = MemoryClient(api_key=mem0_api_key)


        # Add conversation to cloud storage with metadata
        result = client.add(
            sample_messages, user_id=user_id, metadata={"category": "cloud_movie_preferences", "session": "cloud_demo"},version="v2"
        )
        print(f"Add result: {result}")

        # Add preferences sequentially to cloud
        for i, preference in enumerate(sample_preferences[:3]):  # Limit for demo
            # Convert string preference to message format
            preference_message = [{"role": "user", "content": preference}]
            result = client.add(preference_message, user_id=user_id, metadata={"type": "cloud_preference", "index": i})
        

        # 2. SEARCH operations - leverage cloud search capabilities
        search_result = client.search("What are the user's movie preferences?", user_id=user_id)
        print(f"Search result: {search_result}")

        # 3. GET_ALL - retrieve all memories for the user
        all_memories = client.get_all(user_id=user_id, limit=10)
        print(f"Cloud memories retrieved: {all_memories}")

        # Cleanup - remove all user memories from cloud
        delete_all_result = client.delete_all(user_id=user_id)
        print(f"Delete all result: {delete_all_result}")
        agentops.end_trace(end_state="success")
    except Exception as e:
        agentops.end_trace(end_state="error")

## Asynchronous Memory Operations

This function showcases the power of asynchronous operations with Mem0's AsyncMemoryClient. By using async/await patterns, we can execute multiple memory operations concurrently, significantly improving performance.


In [100]:
async def demonstrate_async_memory_client(sample_messages, sample_preferences, user_id):
    """
    Demonstrate asynchronous MemoryClient operations with concurrent cloud access.
    
    This function performs concurrent cloud memory operations including:
    - Initializing async cloud-based memory client
    - Adding multiple memories concurrently using asyncio.gather()
    - Performing parallel search operations across cloud storage
    - Retrieving filtered memories asynchronously
    - Cleaning up cloud memories efficiently
    
    """
    agentops.start_trace("mem0_memoryclient_async_example",tags=["mem0_memoryclient_example"])
    try:
        # Initialize async MemoryClient for concurrent cloud operations
        async_client = AsyncMemoryClient(api_key=mem0_api_key)
    
        # Add conversation and preferences concurrently to cloud
        add_conversation_task = async_client.add(
            sample_messages, user_id=user_id, metadata={"category": "async_cloud_movies", "session": "async_cloud_demo"}
        )

        # Create tasks for adding preferences in parallel
        add_preference_tasks = [
            async_client.add([{"role": "user", "content": pref}], user_id=user_id, metadata={"type": "async_cloud_preference", "index": i})
            for i, pref in enumerate(sample_preferences[:3])
        ]

        # Execute all add operations concurrently
        results = await asyncio.gather(add_conversation_task, *add_preference_tasks)
        for i, result in enumerate(results):
            print(f"{i+1}. {result}")

        # 2. Concurrent SEARCH operations - multiple cloud searches in parallel
        search_tasks = [
            async_client.search("movie preferences", user_id=user_id),
            async_client.search("food preferences", user_id=user_id),
            async_client.search("work information", user_id=user_id),
        ]

        # Execute all searches concurrently
        search_results = await asyncio.gather(*search_tasks)
        for i, result in enumerate(search_results):
            print(f"Search {i+1} result: {result}")

        # 3. GET_ALL operation - retrieve all memories from cloud
        all_memories = await async_client.get_all(user_id=user_id, limit=10)
        print(f"Async cloud memories: {all_memories}")

        # Final cleanup - remove all memories asynchronously
        delete_all_result = await async_client.delete_all(user_id=user_id)
        print(f"Delete all result: {delete_all_result}")

        agentops.end_trace(end_state="success")

    except Exception as e:
        agentops.end_trace(end_state="error")

## Execute Demonstrations

Run both synchronous and asynchronous demonstrations to compare their behavior and performance. The async version typically completes faster due to concurrent operations, especially when dealing with multiple API calls.


In [101]:
# Execute both sync and async demonstrations
demonstrate_sync_memory_client(sample_messages, sample_preferences, user_id)
await demonstrate_async_memory_client(sample_messages, sample_preferences, user_id)

🖇 AgentOps: Session Replay for mem0_memoryclient_sync_example trace: https://app.agentops.ai/sessions?trace_id=75c9c672ca2f2205d1bdb9b4f615ba2e


args: ([{'role': 'user', 'content': "I'm planning to watch a movie tonight. Any recommendations?"}, {'role': 'assistant', 'content': 'How about a thriller? They can be quite engaging.'}, {'role': 'user', 'content': "I'm not a big fan of thriller movies but I love sci-fi movies."}, {'role': 'assistant', 'content': "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future."}],)
kwargs: {'user_id': 'alice_demo', 'metadata': {'category': 'cloud_movie_preferences', 'session': 'cloud_demo'}, 'version': 'v2'}
return_value: None


/Users/fenil/Documents/agentops/venv/lib/python3.11/site-packages/mem0/client/main.py:34: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default.Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


args: None
kwargs: None
return_value: {'results': [{'id': '15b4579e-a074-4ef6-a318-03078a18858b', 'event': 'ADD', 'memory': 'Prefers sci-fi movies over thriller movies'}]}
Add result: {'results': [{'id': '15b4579e-a074-4ef6-a318-03078a18858b', 'event': 'ADD', 'memory': 'Prefers sci-fi movies over thriller movies'}]}
args: ([{'role': 'user', 'content': 'I prefer dark roast coffee over light roast'}],)
kwargs: {'user_id': 'alice_demo', 'metadata': {'type': 'cloud_preference', 'index': 0}}
return_value: None
args: None
kwargs: None
return_value: {'results': [{'id': 'd012318e-6dd3-4456-9ddb-1e33b5973201', 'event': 'ADD', 'memory': 'Prefers dark roast coffee over light roast'}]}
args: ([{'role': 'user', 'content': 'I exercise every morning at 6 AM'}],)
kwargs: {'user_id': 'alice_demo', 'metadata': {'type': 'cloud_preference', 'index': 1}}
return_value: None
args: None
kwargs: None
return_value: {'results': [{'id': '1e0829f9-ada4-4fde-8371-9a0b24410692', 'event': 'ADD', 'memory': 'Exercises 

🖇 AgentOps: Session Replay for mem0_memoryclient_sync_example.session trace: https://app.agentops.ai/sessions?trace_id=75c9c672ca2f2205d1bdb9b4f615ba2e
🖇 AgentOps: Session Replay for mem0_memoryclient_async_example trace: https://app.agentops.ai/sessions?trace_id=077cb3ff917063d6c4c668dd3de1a5ed


args: ([{'role': 'user', 'content': "I'm planning to watch a movie tonight. Any recommendations?"}, {'role': 'assistant', 'content': 'How about a thriller? They can be quite engaging.'}, {'role': 'user', 'content': "I'm not a big fan of thriller movies but I love sci-fi movies."}, {'role': 'assistant', 'content': "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future."}],)
kwargs: {'user_id': 'alice_demo', 'metadata': {'category': 'async_cloud_movies', 'session': 'async_cloud_demo'}}
return_value: None
args: None
kwargs: None
return_value: <coroutine object AsyncMemoryClient.add at 0x117000040>
args: ([{'role': 'user', 'content': 'I prefer dark roast coffee over light roast'}],)
kwargs: {'user_id': 'alice_demo', 'metadata': {'type': 'async_cloud_preference', 'index': 0}}
return_value: None
args: None
kwargs: None
return_value: <coroutine object AsyncMemoryClient.add at 0x117001be0>
args: ([{'role': 'user', 'content': 'I exercise every morning at 6 AM'}],)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/events.py:84: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default.Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  self._context.run(self._callback, *self._args)


1. {'results': [{'id': '2ef3c2da-4ad1-4b5d-b735-9852d78c17e7', 'event': 'ADD', 'memory': 'Prefers sci-fi movies over thriller movies'}]}
2. {'results': [{'id': '05b6099e-2269-4ba1-9c8c-476a160e180d', 'event': 'ADD', 'memory': 'Prefers dark roast coffee over light roast'}]}
3. {'results': [{'id': '38b92272-b8f2-491a-8674-2b37e60fe93a', 'event': 'ADD', 'memory': 'Exercises every morning at 6 AM'}]}
4. {'results': [{'id': 'e053d1bd-a584-4a82-ba61-eb2b00f299c8', 'event': 'ADD', 'memory': 'Is vegetarian'}, {'id': '2dcee7ea-0684-4ddc-bacd-8a66609649f9', 'event': 'ADD', 'memory': 'Avoids all meat products'}]}
get_search_attributes args: ('movie preferences',)
get_search_attributes kwargs: {'user_id': 'alice_demo'}
get_search_attributes return_value: None
get_search_attributes args: None
get_search_attributes kwargs: None
get_search_attributes return_value: <coroutine object AsyncMemoryClient.search at 0x116fa6560>
get_search_attributes args: ('food preferences',)
get_search_attributes kwargs:

🖇 AgentOps: Session Replay for mem0_memoryclient_async_example.session trace: https://app.agentops.ai/sessions?trace_id=077cb3ff917063d6c4c668dd3de1a5ed
